In [ ]:
import zipfile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

# Loadin csv files

In [ ]:
# READ THE CSV FILES
train_df = pd.read_csv('../input/sentiment-analysis-on-movie-reviews/train.tsv.zip', sep='\t')
test_df = pd.read_csv('../input/sentiment-analysis-on-movie-reviews/test.tsv.zip', sep='\t')

In [ ]:
train_df.head()

In [ ]:
train_df['Sentiment'].value_counts().plot(kind='bar')

In [ ]:
seq_len = 512
num_samples = len(train_df)

num_samples, seq_len

# Tokenizing all Phrases using BertTokenizer

In [ ]:
from transformers import BertTokenizer

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [ ]:
tokens = tokenizer(train_df['Phrase'].tolist(),max_length=seq_len,
                  truncation=True,padding='max_length',
                   add_special_tokens=True,
                   return_tensors='np'
                  )

In [ ]:
tokens.keys()

In [ ]:
tokens['input_ids']

In [ ]:
tokens['attention_mask']

# Saving Input IDs and attention Masks in numpy binary files

In [ ]:
# Save as numpy binary files
with open('movie-xids.npy','wb') as f:
    np.save(f,tokens['input_ids'])
with open('movie-xmask.npy','wb') as f:
    np.save(f,tokens['attention_mask'])

**One hot encoding the target values**

In [ ]:
arr = train_df['Sentiment'].values
print(arr.shape)
print(arr)
print(f"one hot encoding - {arr.max() + 1}")

In [ ]:
labels = np.zeros((num_samples,arr.max()+1))
print(f"shape of labels - {labels.shape}")

In [ ]:
labels[np.arange(num_samples),arr] = 1
print(labels)

In [ ]:
with open('movie-labels.npy','wb') as f:
    np.save(f,labels)

In [ ]:
labels[9]

**Loading the inputids,masks and labes from numpy binary files**

In [ ]:
with open('movie-xids.npy','rb') as f:
    Xids = np.load(f,allow_pickle=True)
with open('movie-xmask.npy','rb') as f:
    Xmask = np.load(f,allow_pickle=True)
with open('movie-labels.npy','rb') as f:
    labels = np.load(f,allow_pickle=True)

In [ ]:
Xids.shape

**Making Tesnorflow dataset adding batch size and shufling**

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((Xids,Xmask,labels))

In [ ]:
dataset.take(1)

In [ ]:
def map_func(input_ids,masks,labels):
    return {'input_ids':input_ids,
           'attention_mask':masks}, labels

In [ ]:
dataset = dataset.map(map_func)

In [ ]:
dataset.take(1)

In [ ]:
batch_size = 16
dataset = dataset.shuffle(10000).batch(batch_size, drop_remainder=True)
dataset.take(1)

In [ ]:
split = 0.9

In [ ]:
print(int((Xids.shape[0]/batch_size) * split))
size = int((Xids.shape[0]/batch_size) * split)

In [ ]:
train_ds = dataset.take(size)
val_ds = dataset.skip(size)

In [ ]:
tf.data.experimental.save(train_ds,'train')
tf.data.experimental.save(val_ds,'val')

In [ ]:
train_ds.element_spec

In [ ]:
train_ds.element_spec == val_ds.element_spec

In [ ]:
ds = tf.data.experimental.load('train',element_spec=train_ds.element_spec)
ds.take(1)

In [ ]:
from transformers import TFAutoModel

In [ ]:
bert = TFAutoModel.from_pretrained('bert-base-cased')

In [ ]:
bert.summary()

**Defining the BERT Model**

In [ ]:
input_ids = tf.keras.layers.Input(shape=(512,),
                                 name="input_ids",dtype='int32')
mask = tf.keras.layers.Input(shape=(512,),name="attention_mask",dtype='int32')

# transformer
embeddings = bert.bert(input_ids,attention_mask=mask)[1]
# classifier head
x = tf.keras.layers.Dense(1024,activation='relu')(embeddings)
y = tf.keras.layers.Dense(5,activation='softmax',name='outputs')(x)
model = tf.keras.Model(inputs=[input_ids,mask],outputs=y)

In [ ]:
model.layers[2].trainable = False

In [ ]:
model.summary()

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5,decay=1e-6)
loss = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer,loss=loss, metrics=[acc,])

In [ ]:
element_spec = ({'input_ids': tf.TensorSpec(shape=(16, 512), dtype=tf.int64, name=None),
  'attention_mask': tf.TensorSpec(shape=(16, 512), dtype=tf.int64, name=None)},
 tf.TensorSpec(shape=(16, 5), dtype=tf.float64, name=None))

In [ ]:
train_ds = tf.data.experimental.load('train',element_spec=element_spec)

In [ ]:
val_ds = tf.data.experimental.load('val',element_spec=element_spec)

In [ ]:
train_ds.take(1)

**Training the model**

In [ ]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=3
)

In [ ]:
model.save('sentiment_model')

In [ ]:
model = tf.keras.models.load_model('sentiment_model')

# Making Predictions

In [ ]:
def prep_data(text):
    tokens = tokenizer(text,max_length=512,truncation=True,
                      padding='max_length',add_special_tokens=True,
                      return_tensors='tf')
    return {"input_ids": tokens['input_ids'],
           "attention_mask": tokens["attention_mask"]}

In [ ]:
probs = model.predict(prep_data('hello world'))[0]

In [ ]:
np.argmax(probs)

In [ ]:
probs = model.predict(prep_data('this movie was amazing'))[0]

In [ ]:
np.argmax(probs)

In [ ]:
probs = model.predict(prep_data('this movie was terrible'))[0]
np.argmax(probs)

In [ ]:
pd.set_option('display.max_colwidth',None)

In [ ]:
test_df.head()

In [ ]:
df = test_df.drop_duplicates(subset=['SentenceId'],keep='first')
df.head()

In [ ]:
df['Sentiment'] = None

In [ ]:
for i, row in df.iterrows():
    tokens = prep_data(row['Phrase'])
    probs = model.predict(tokens)
    pred = np.argmax(probs)
    df.at[i,'Sentiment'] = pred

In [ ]:
df.head()

In [ ]:
df.tail()